<a href="https://colab.research.google.com/github/desakim/ESAA_OB/blob/main/desakim_ob0922_handson245_271.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

기한: 9월 22일 (금) 18:30

과제: 핸즈온 7장 앙상블 학습과 랜덤 포레스트

p. 245-271 필사하여 깃허브에 업로드한 뒤, 깃허브 링크를 댓글로 달아주세요.

#Chapter 7. 앙상블 학습과 랜덤 포레스트

## 7.1 투표 기반 분류기



In [1]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr',log_clf), ('rf',rnd_clf), ('svc',svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [3]:
#각 분류기의 테스트셋 정확도
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.904


- 간접 투표 : voting="hard" 를 voting="soft" 로 바꾸고 모든 분류기가 클래스의 확률을 추정할 수 있으면 됨. 즉 predict_proba() 메서드가 있으면 됨.

## 7.2 배깅과 페이스팅

* 배깅 : 훈련 세트에서 중복을 허용하여 샘플링하는 방식
* 페이스팅 : 중복을 허용하지 않고 샘플링 하는 방식

### 7.2.1 사이킷런의 배깅와 페이스팅


In [4]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1)  # n_jobs: 사이킷런이 훈련과 예측에 사용할 CPU 코어 수 지정(-1로 지저앟면 가용한 모든 코어 사용)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

### 7.2.2 oob 평가

배깅을 사용하면 어떤 샘플은 한 예측기를 위해 여러 번 샘플링되고 어떤 것은 전혀 선택되지 않을 수 있음.

BaggingClassifier는 기본값으로 중복을 허용하여 훈련 세트의 크기인 m개의 샘플을 선택.

평균적으로 각 예측기에 훈련 샘플의 63% 정도만 샘플링 된다는 것을 의미.

선택되지 않은 나머지 37%를 oob(out-of-bag) 샘플이라고 부름.

* 사이킷런에서 BaggingClassifier 만들 때 oob_score=True로 지정하면 훈련 끝난 후 자동으로 oob 평가 수행. 이는 obb_score 변수에 저장

In [5]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.896

In [6]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.896

In [7]:
bag_clf.oob_decision_function_

array([[0.36407767, 0.63592233],
       [0.30939227, 0.69060773],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.09090909, 0.90909091],
       [0.30243902, 0.69756098],
       [0.01086957, 0.98913043],
       [1.        , 0.        ],
       [0.98255814, 0.01744186],
       [0.79881657, 0.20118343],
       [0.01142857, 0.98857143],
       [0.78894472, 0.21105528],
       [0.81443299, 0.18556701],
       [0.95833333, 0.04166667],
       [0.06735751, 0.93264249],
       [0.        , 1.        ],
       [0.98907104, 0.01092896],
       [0.92903226, 0.07096774],
       [0.98974359, 0.01025641],
       [0.01104972, 0.98895028],
       [0.36125654, 0.63874346],
       [0.91457286, 0.08542714],
       [1.        , 0.        ],
       [0.98742138, 0.01257862],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.61878453, 0.38121547],
       [0.

##7.3 랜덤 패치와 랜덤 서브스페이스

BaggingClassifier는 특성 샘플링도 지원.

* 샘플링은 두 매개 변수로 조정: mac_features, bootstrap_features

고차원 데이터셋을 다룰 때 유용.
* 랜덤 패치 방식: 훈련 특성과 샘플을 모두 샘플링

* 랜덤 서브스페이스 방식: 훈련 샘플을 모두 사용(bootstrao=False, max_samples=1.0)하고 특성은 샘플링(bootstrao=True, max_samples=1.0보다 작게 설정)하는 것.

##7.4 랜덤 포레스트

max_samples 를 훈련 세트의 크기로 지정.

결정 트리에 최적화되어 사용하기 편리한 RandomForesetClassifier 사용가능.

In [8]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [9]:
# BaggingClassifier 사용해 FandomForestClassifier와 거의 유사하게 만듦

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)

###7.4.1 엑스트라 트리

### 7.4.2 특성 중요도



In [10]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.08549164665017488
sepal width (cm) 0.025035399284005735
petal length (cm) 0.4552574069865528
petal width (cm) 0.43421554707926663


##7.5 부스팅

###7.5.1 에이다부스트



In [11]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

###7.5.2 그레이디언트 부스팅



In [12]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2)

In [13]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2)

In [14]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2)

In [16]:
# 세 개의 트리를 포함하는 앙상블 모델의 예측을 더하면 새로운 샘플에 대한 예측
#y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [17]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [18]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X,y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=106)

In [19]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
  gbrt.n_estimators = n_estimators
  gbrt.fit(X_train, y_train)
  y_pred = gbrt.predict(X_val)
  val_error = mean_squared_error(y_val, y_pred)
  if val_error < min_val_error:
    min_val_error = val_error
    error_going_up = 0
  else:
    error_going_up += 1
    if error_going_up == 5:
      break    # 조기 종료

In [20]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

In [21]:
#XGBoost 자동 조기 종료 기능
xgb_reg.fit(X_train, y_train,
            eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.41212
[1]	validation_0-rmse:0.36620
[2]	validation_0-rmse:0.33287
[3]	validation_0-rmse:0.32179
[4]	validation_0-rmse:0.31342
[5]	validation_0-rmse:0.31461
[6]	validation_0-rmse:0.31547


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


##7.6 스태킹

